In [1]:
import uuid
import pandas
import numpy
from random import shuffle

import sklearn
import sklearn.ensemble

from sklearn.ensemble import RandomForestRegressor

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import os

In [2]:
from sklearn import preprocessing
import numpy

In [3]:
def load_data():
    
    filename = "/Users/simondi/PHD/data/data/target_infect_x/screening_data_subset/cells_sample_10_normalized_cut.tsv"

    data = pandas.read_csv(filename, sep="\t", header=0)

    feature_cols = [(i, x) for i, x in enumerate(data.columns.values) if
                    x.startswith("cells")]
    for i, c in feature_cols:
        data.loc[:, c] = data.loc[:, c].astype('float64')

    X = data.dropna(subset=["cells.children_invasomes_count", "cells.children_bacteria_count"], how='all')
    X["cells.children_invasomes_count"] =  X["cells.children_invasomes_count"].fillna(0)
    X["cells.children_bacteria_count"] =  X["cells.children_bacteria_count"].fillna(0)
    X = X.dropna()
    X["infection"] = X["cells.children_invasomes_count"] + X["cells.children_bacteria_count"] 
    del X["cells.children_invasomes_count"]
    del X["cells.children_bacteria_count"]
    return X

In [21]:
data = load_data()

/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
data.groupby(["pathogen", "gene"]).size()

pathogen    gene    
bartonella  a1bg         20
            a1cf         60
            a2ld1        60
            a2m          60
            a2ml1        30
            a4galt       50
            a4gnt        60
            aaa1         40
            aaas         60
            aacs         50
            aacsp1       30
            aadac        60
            aadacl2      50
            aadacl4      40
            aadat        60
            aagab        60
            aak1        160
            aamp         60
            aanat        60
            aars         60
            aars2        60
            aasdh        60
            aasdhppt     60
            aass         60
            aatf         50
            aatk        120
            aatk-as1     40
            abat         60
            abca1        60
            abca10       40
                       ... 
salmonella  zpld1        30
            zranb1       30
            zranb2       30
            zranb3       30

In [24]:
df = data.groupby(["pathogen", "gene"]).apply(lambda x: x.sample(frac=0.1))

In [25]:
df.groupby(["pathogen", "gene"]).size()

/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'pathogen' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  if __name__ == '__main__':
/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'gene' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  if __name__ == '__main__':


pathogen    gene    
bartonella  a1bg         2
            a1cf         6
            a2ld1        6
            a2m          6
            a2ml1        3
            a4galt       5
            a4gnt        6
            aaa1         4
            aaas         6
            aacs         5
            aacsp1       3
            aadac        6
            aadacl2      5
            aadacl4      4
            aadat        6
            aagab        6
            aak1        16
            aamp         6
            aanat        6
            aars         6
            aars2        6
            aasdh        6
            aasdhppt     6
            aass         6
            aatf         5
            aatk        12
            aatk-as1     4
            abat         6
            abca1        6
            abca10       4
                        ..
salmonella  zpld1        3
            zranb1       3
            zranb2       3
            zranb3       3
            zscan1       3
       

In [26]:
n = 11
rf = RandomForestRegressor(n_estimators=n, max_features='sqrt')

In [27]:
feature_cols = [x for x in df.columns.values if x.startswith("cells")]
trees=rf.fit(X=df.loc[:,feature_cols], y=df.loc[:,"infection"])

In [28]:
trees.score(X=df.loc[:,feature_cols], y=df.loc[:,"infection"])

0.83353090460555201

In [30]:
df_tet = data.groupby(["pathogen", "gene"]).apply(lambda x: x.sample(frac=0.01))
trees.score(X=df_tet.loc[:,feature_cols], y=df_tet.loc[:,"infection"])

0.13709412625453066

In [11]:
zippedf = [(x,y) for x, y in zip(feature_cols, trees.feature_importances_)]

In [18]:
zippedf = sorted(zippedf, key=lambda x: -x[1]) 

In [19]:
zippedf

[('cells.areashape_area', 0.097864321068438884),
 ('cells.areashape_majoraxislength', 0.088423111784056804),
 ('cells.areashape_perimeter', 0.082355533746508947),
 ('cells.areashape_minoraxislength', 0.072936919991272359),
 ('cells.areashape_extent', 0.057534282248412942),
 ('cells.neighbors_secondclosestxvector_2', 0.056293865425443651),
 ('cells.neighbors_secondclosestyvector_2', 0.050153754949301209),
 ('cells.neighbors_numberofneighbors_2', 0.049989290523752006),
 ('cells.neighbors_firstclosestyvector_2', 0.049014556008228564),
 ('cells.areashape_formfactor', 0.047766145405209411),
 ('cells.neighbors_anglebetweenneighbors_2', 0.047738924780533806),
 ('cells.neighbors_firstclosestxvector_2', 0.045506902534460276),
 ('cells.areashape_eccentricity', 0.04511720739735206),
 ('cells.location_center_x', 0.043952101681158692),
 ('cells.location_center_y', 0.043693306026592556),
 ('cells.parent_nuclei', 0.042436404911890088),
 ('cells.neighbors_firstclosestobjectnumber_2', 0.041494806615221

In [15]:
uid = str(uuid.uuid1())
flout= "/Users/simondi/PROJECTS/target_infect_x_project/src/tix_util/tix_analysis/results/importance"

In [24]:
flout = flout + "_trees_" + str(n)+ "_" + uid + ".txt"

In [27]:
with open(flout, 'w') as fh:
    for z in zippedf:
        fh.write(z[0] +"\t"  + "{0:.2f}".format(z[1]) + "\n")